In [1]:
import anndata as ad
import scanpy as sc
import numpy as np
from adata_utils import fix_genes_chunked

In [2]:
mcfaline = ad.read_h5ad("/mnt/hdd2/tan/perturbench/mcfaline23_gxe_preprocessed.h5ad")

In [3]:
mcfaline.layers["counts"]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1128972813 stored elements and shape (878229, 15009)>

In [4]:
sc.pp.normalize_total(mcfaline, target_sum=1e4, layer="counts")
sc.pp.log1p(mcfaline, layer="counts")

In [5]:
gene_names = open("./gene_names.csv").read().splitlines()
len(gene_names), gene_names[:5]

(18080, ['SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'PERM1'])

In [6]:
mcfaline.X = mcfaline.layers["counts"]

In [7]:
adata = fix_genes_chunked(mcfaline, gene_names)

Processing chunk 0/87
Processing chunk 1/87
Processing chunk 2/87
Processing chunk 3/87
Processing chunk 4/87
Processing chunk 5/87
Processing chunk 6/87
Processing chunk 7/87
Processing chunk 8/87
Processing chunk 9/87
Processing chunk 10/87
Processing chunk 11/87
Processing chunk 12/87
Processing chunk 13/87
Processing chunk 14/87
Processing chunk 15/87
Processing chunk 16/87
Processing chunk 17/87
Processing chunk 18/87
Processing chunk 19/87
Processing chunk 20/87
Processing chunk 21/87
Processing chunk 22/87
Processing chunk 23/87
Processing chunk 24/87
Processing chunk 25/87
Processing chunk 26/87
Processing chunk 27/87
Processing chunk 28/87
Processing chunk 29/87
Processing chunk 30/87
Processing chunk 31/87
Processing chunk 32/87
Processing chunk 33/87
Processing chunk 34/87
Processing chunk 35/87
Processing chunk 36/87
Processing chunk 37/87
Processing chunk 38/87
Processing chunk 39/87
Processing chunk 40/87
Processing chunk 41/87
Processing chunk 42/87
Processing chunk 43/8

In [8]:
adata.obs = mcfaline.obs

In [10]:
adata

AnnData object with n_obs × n_vars = 878229 × 18080
    obs: 'orig.ident', 'ncounts', 'ngenes', 'cell', 'sample', 'Size_Factor', 'n.umi', 'PCR_plate', 'new_cell', 'dose', 'treatment', 'gRNA_id', 'gene_id', 'guide_number', 'cell_type', 'drug_dose', 'perturbation_type', 'dataset', 'gene_dose', 'perturbation', 'pert_cl_tr', 'condition', 'condition_plus_treatment', 'cov_merged', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'control', 'dose_val', 'cov_drug_dose_name'
    var: 'ensembl_id', 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'

In [11]:
target_gene = adata.obs["gene_id"].tolist()
for i, val in enumerate(target_gene):
    vals = [val, "non-targeting"]
    target_gene[i] = vals[int(val == "control")]
adata.obs["target_gene"] = target_gene

In [12]:
adata.obs["batch_var"] = adata.obs["PCR_plate"]

In [13]:
adata.obs.cell_type.unique()

['a172', 't98g', 'u87mg']
Categories (3, object): ['a172', 't98g', 'u87mg']

In [16]:
adata.var.drop(columns=["highly_variable"], inplace=True)

In [17]:
data_dir = "/mnt/hdd2/tan/competition_support_set_sorted/"
for cell_type in adata.obs.cell_type.unique():
    print("Processing cell type: ", cell_type)
    ct_adata = adata[adata.obs.cell_type == cell_type, ]
    ct_adata = ct_adata[ct_adata.obs.target_gene.sort_values().index, ]
    ct_adata.write_h5ad(f"{data_dir}/mcfaline_{cell_type}.h5")

Processing cell type:  a172


/mnt/hdd1/tan/conda/miniconda3/envs/python312/lib/python3.12/site-packages/anndata/_core/anndata.py:1175: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/mnt/hdd1/tan/conda/miniconda3/envs/python312/lib/python3.12/site-packages/anndata/_core/anndata.py:1175: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


Processing cell type:  t98g


/mnt/hdd1/tan/conda/miniconda3/envs/python312/lib/python3.12/site-packages/anndata/_core/anndata.py:1175: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/mnt/hdd1/tan/conda/miniconda3/envs/python312/lib/python3.12/site-packages/anndata/_core/anndata.py:1175: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


Processing cell type:  u87mg


/mnt/hdd1/tan/conda/miniconda3/envs/python312/lib/python3.12/site-packages/anndata/_core/anndata.py:1175: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/mnt/hdd1/tan/conda/miniconda3/envs/python312/lib/python3.12/site-packages/anndata/_core/anndata.py:1175: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


In [18]:
adata = ad.read_h5ad("/mnt/hdd2/tan/competition_support_set_sorted/mcfaline_a172.h5", "r")
adata.obs.target_gene

06B_C06_RT_BC_318_Lig_BC_173             AAK1
03A_G03_RT_BC_317_Lig_BC_146             AAK1
10A_H10_RT_BC_212_Lig_BC_297             AAK1
11A_A11_RT_BC_95_Lig_BC_341              AAK1
11B_C11_RT_BC_187_Lig_BC_254             AAK1
                                    ...      
02C_B02_RT_BC_359_Lig_BC_10     non-targeting
08B_D08_RT_BC_363_Lig_BC_351    non-targeting
02C_B02_RT_BC_35_Lig_BC_194     non-targeting
08B_D08_RT_BC_374_Lig_BC_63     non-targeting
08B_D08_RT_BC_34_Lig_BC_93      non-targeting
Name: target_gene, Length: 302826, dtype: category
Categories (529, object): ['AAK1', 'AATK', 'ABL1', 'ABL2', ..., 'YES1', 'ZAK', 'ZAP70', 'non-targeting']